In [ ]:
# import pandas as pd

# prices_path = r"C:\_projekty_repo\IESW25\da3\data\weather_hourly.csv"

# df_prices = pd.read_csv(
#     prices_path,
#     index_col=0,
#     parse_dates=[0],
#     date_format="%Y%m%d:%H%M"  # matches "20050101:0010"
# )

# # convert all to floats
# df_prices = df_prices.astype(float)

# df_prices.index = df_prices.index.round("15min")

# df_prices_15minresampled = df_prices.resample("15min").interpolate(method="linear")
# df_prices_15minresampled
# df_prices_15minresampled.to_csv(r"C:\_projekty_repo\IESW25\da3\data\weather_15min.csv", index=True)

In [ ]:
# import pandas as pd
# import numpy as np

# #["elec_price_EUR_per_kWh", "gas_price_EUR_per_kWh"]
# path_prices = r"C:\_projekty_repo\IESW25\da3\data\prices_15min.csv"

# df_prices = pd.read_csv(path_prices,index_col=0,parse_dates=True)
# #rename Market prices - Day Ahead - Forecast to energy_prices
# df_prices.columns = ["elec_price_EUR_per_MWh"]
# df_prices_daily_mean_med = df_prices.resample("D").agg(
#     daily_mean=("elec_price_EUR_per_MWh", "mean"),
#     daily_median=("elec_price_EUR_per_MWh", "median")
# )

# # take the maximum of daily mean and median
# base_price = df_prices_daily_mean_med[["daily_mean", "daily_median"]].max(axis=1)

# # model gas price: electricity is ~2x gas + allowance
# # → gas = base / (2 + allowance + random noise)
# allowance = 0.2   # tweak this value as you like
# random_noise = np.random.uniform(-0.1, 0.1, size=len(base_price))  # ±0.1 variation

# df_prices_daily_mean_med["gas_price_EUR_per_MWh"] = (
#     base_price / (2 + allowance + random_noise)
# )

# # --- 🔑 Merge daily gas price into 15-min dataframe ---
# df_prices["gas_price_EUR_per_MWh"] = (
#     df_prices_daily_mean_med["gas_price_EUR_per_MWh"]
#     .reindex(df_prices.index, method="ffill")  # forward-fill daily to intraday
# )

# df_prices.to_csv(r"C:\_projekty_repo\IESW25\da3\data\prices_15min.csv", index=True)
# df_prices

In [ ]:
# --- Cell 0A: Weather & price data procurement (load-or-synthesize) ---

import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import date
from demandlib import bdew
import holidays



YEAR = 2024  # choose your simulation year (note: 2024 is leap year)
FREQ = "15min"

# Dummy paths you can swap for real datasets later
PATH_WEATHER = Path("data/weather_15min.csv")   # expected cols: time, irradiance_Wm2, ambient_temp_C
PATH_PRICES  = Path("data/prices_15min.csv")    # expected cols: time, datetime,elec_price_EUR_per_MWh,gas_price_EUR_per_MWh


def load_weather_prices() -> pd.DataFrame:

    # ---- weather ----
    if PATH_WEATHER.exists():
        print("Loading weather data...")
        print(f"From: {PATH_WEATHER.resolve()}")
        w = pd.read_csv(PATH_WEATHER, index_col=0, parse_dates=True)        # if any columns missing, backfill synthetic
        w_missing = []
        for c in ["P[W]","irradiance_Wm2", "ambient_temp_C"]:
            if c not in w.columns:
                w_missing.append(c)
        if w_missing:
            print(f"Missing weather columns: {w_missing}, synthesizing simple model.")
            for c in w_missing:
                w[c] = w_synth[c]
        w = w[["P[W]","irradiance_Wm2", "ambient_temp_C"]]
        w["P[kWh]"] = w["P[W]"] * (15/60/1000)   # convert W to kWh for the 15min step
    else:
        print("No weather data found, synthesizing simple model.")

    # ---- prices ----
    if PATH_PRICES.exists():
        print("Loading price data...")
        print(f"From: {PATH_PRICES.resolve()}")
        p = pd.read_csv(PATH_PRICES,index_col=0,parse_dates=True)
        p_missing = []
        for c in ["elec_price_EUR_per_MWh", "gas_price_EUR_per_MWh"]:
            if c not in p.columns:
                p_missing.append(c)
        if p_missing:
            print(f"Missing price columns: {p_missing}, synthesizing simple model.")
            for c in p_missing:
                p[c] = p_synth[c]
        p = p[["elec_price_EUR_per_MWh", "gas_price_EUR_per_MWh"]]
    else:
        print("No price data found, synthesizing simple model.")


    return p, w

    




In [ ]:
p, w = load_weather_prices()

p_no_dups = p[~p.index.duplicated(keep='first')]
w_no_dups = w[~w.index.duplicated(keep='first')]



df_external_data = pd.concat([w_no_dups, p_no_dups], axis=1)

# Create a full 15-min index between min and max
full_index = pd.date_range(
    start=df_external_data.index.min(),
    end=df_external_data.index.max(),
    freq="15min"
)

# Reindex to include all expected timestamps
df_external_data = df_external_data.reindex(full_index)

# Interpolate linearly in time
df_external_data = df_external_data.interpolate(method="time")

# Fill any remaining leading/trailing NaNs
df_external_data = df_external_data.ffill().bfill()


#sfh.pre_inspect(df_external_data, start="2022-02-01", end="2022-02-14",return_fig=False)
start_inspect = "2022-02-01"
end_inspect = "2022-02-14"
import plotly.express as px

fig = px.line(
    df_external_data[start_inspect:end_inspect],
    title=f"External Data from {start_inspect} to {end_inspect}"
)
fig.show()

In [ ]:
import holidays


holidays_de = holidays.country_holidays('DE', years=[YEAR])
holidays = {d: holidays_de[d] for d in holidays_de}
holidays_de
# --- Heat subjects with descriptive names ---
heat_subjects = {
    "efh": "single_family_house",
    "mfh": "multi_family_house",
    "gmk": "metal_automotive",
    "gha": "retail_wholesale",
    "gko": "authorities_finance_insurance",
    "gbd": "other_services",
    "gga": "restaurants",
    "gbh": "accommodation",
    "gwa": "laundries_drycleaning",
    "ggb": "horticulture",
    "gba": "bakery",
    "gpd": "paper_printing",
    "gmf": "household_like_business",
    "ghd": "commerce_services_total",
}
idx = pd.date_range("2021-01-01 00:00", "2023-12-31 23:00", freq="15min")
df_heat = pd.DataFrame(index=idx)
year_count = sorted(set(idx.year))
for code, name in heat_subjects.items():
    params = dict(building_class=5) if code in ["efh", "mfh"] else dict(building_class=0)
    hb = bdew.HeatBuilding(
        df_index=idx,
        temperature=df_external_data["ambient_temp_C"].reindex(idx, method="ffill"),
        holidays=holidays,
        wind_class=1,
        climate_region=5,
        shlp_type=code.upper(),   # demandlib expects EFH, MFH, etc.
        **params,
    )

    
    df_heat[name] = hb.get_normalized_bdew_profile()
    #multiply by year count
    df_heat[name] = df_heat[name] * len(year_count)
# # --- Electrical profiles ---
# e_slp = bdew.ElecSlp(year=2024, holidays=holidays, df_index=idx)
# df_elec = e_slp.get_profiles()

# # rename with lowercase + descriptive names


# Multi-year 15-min index (use 23:45 for inclusive daily coverage)
idx = pd.date_range("2021-01-01 00:00", "2023-12-31 23:45", freq="15min")

# Optional: holidays per year (list/dict of datetimes). Example empty:
hol_by_year = {y: [] for y in sorted(set(idx.year))}

parts = []
for y in sorted(set(idx.year)):
    chunk = idx[idx.year == y]
    e = bdew.ElecSlp(year=y, holidays=hol_by_year[y])             # ctor has no df_index
    df_y = e.all_load_profiles(chunk, holidays=hol_by_year[y])    # normalized to 1
    parts.append(df_y)

df_elec = pd.concat(parts).sort_index()
elec_map = {
    "h0": "household",
    "h0_dyn": "household_dynamic",
    "g0": "general_commerce",
    "g1": "commerce_office_day",
    "g2": "commerce_evening",
    "g3": "commerce_continuous",
    "g4": "shop_barber",
    "g5": "bakery",
    "g6": "weekend_operation",
    "g7": "mobile_station",
    "l0": "farming_general",
    "l1": "farming_dairy",
    "l2": "farming_other",
}
df_elec = df_elec.rename(columns=elec_map) 
#remame index datetime
df_elec.index.name = "datetime"
df_heat.index.name = "datetime"

In [ ]:
pd.concat([df_external_data.head(2),df_external_data.tail(2)])

In [ ]:
# Existing stats
print("Non-NaNs per column:\n", df_external_data.count())
print("\nNaNs per column:\n", df_external_data.isna().sum())
print("\nTotal non-NaNs:", df_external_data.count().sum())

# --- New: check index continuity ---
expected_freq = "15min"  # or "15T"
expected_range = pd.date_range(
    start=df_external_data.index.min(),
    end=df_external_data.index.max(),
    freq=expected_freq
)

missing_timestamps = expected_range.difference(df_external_data.index)

print(f"\nIndex check: {df_external_data.index.min()} → {df_external_data.index.max()}")
print(f"Expected steps: {len(expected_range)} | Actual steps: {len(df_external_data)}")
print(f"Missing timestamps: {len(missing_timestamps)}")

# Optionally preview the first few missing
if len(missing_timestamps) > 0:
    print("Sample missing:", missing_timestamps[:10])


In [ ]:
print(df_elec.sum())
pd.concat([df_elec.head(2),df_elec.tail(2)])

In [ ]:
print(df_heat.sum())
pd.concat([df_heat.head(2),df_heat.tail(2)])

In [ ]:
# --- SimulationLite: minimal runner for df_external_data (no subjects needed) ---

import pandas as pd
import numpy as np
import time
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

import pandas as pd
import numpy as np
import time
from tqdm import tqdm
# -----------------------------------------------
# consumer_producer_and_simulation.py
# -----------------------------------------------
from __future__ import annotations  
from typing import Dict, Any, List
import time
import numpy as np
import pandas as pd



# What the simulation will persist per subject (flat, easy to grade / plot)
METRICS = {
    "electricity_needed":      "electricity_needed_kW",
    "heat_needed":             "heat_needed_kW",
    "gas_consumed":            "gas_input_kW",
    "electricity_imported":    "grid_kW",
    "electricity_selfconsumed":"selfconsumed_kW",
    "electricity_exported":    "exported_kW",
}

def _get(sig: pd.Series | Dict[str, float], key: str, default: float = 0.0) -> float:
    """Safe access for Series/dict."""
    try:
        return float(sig[key])
    except Exception:
        return default
    




# TODO: Add gas reporting in volume units (m³) as requested: convert fuel kW (power) over timestep to kWh and then to m³ using user-supplied gas calorific value and units, and expose both kW and m³ as outputs.

# TODO: Add explicit round-trip/battery efficiency and separate coulomb/charge/discharge efficiencies (not only power limits). Charge/discharge losses, and idle/self-discharge over time.
# TODO: Track and persist cumulative counters per subject (energy in/out, lifetime throughput, gas m³ consumed, PV generation kWh, exported kWh, imported kWh).
# TODO: Model battery degradation (calendar + cycle) or at minimum cumulative throughput tracking for later replacement cost calculations. #add the counter of the cycles + per cycle degradation (1/total_cycles_lifetime)
# --------------------- Consumer ---------------------
class Consumer:
    """
    Aggregated consumer (e.g., many similar households).
    Inputs:
      - annual_elec_kWh_per_unit / annual_heat_kWh_per_unit
      - nominal_elec_profile / heating_profile: Series aligned to sim index, each normalized to 1.0
    Simple device stack order (heat-first): micro-CHP -> heat pump -> e-heater -> gas boiler
    Electricity balance: demand - PV - CHP +/- battery => grid import/export
    """
    
    def __init__(
        self,
        name: str,
        n_units: int,
        annual_elec_kWh_per_unit: float,
        annual_heat_kWh_per_unit: float,
        nominal_elec_profile: pd.Series,   # normalized to 1 over the sim window
        heating_profile: pd.Series,        # normalized to 1 over the sim window
        # PV
        pv_kWp_per_unit: float = 0.0,
        pv_performance_ratio: float = 0.85,
        # Heat pump (HVAC)
        hvac_max_elec_kW_per_unit: float = 0.0,
        # Resistive heater
        eheater_max_kW_per_unit: float = 0.0,
        # Gas boiler
        gas_boiler_max_kW_per_unit: float = 0.0,
        gas_boiler_efficiency: float = 0.90,
        # Micro-CHP (optional; heat-led)
        chp_elec_cap_kW_per_unit: float = 0.0,
        chp_eta_elec: float = 0.30,
        chp_eta_heat: float = 0.50,
        # Battery (optional)
        battery_capacity_kWh: float = 0.0,
        initial_battery_soc_percent: float = 0.9,   # fraction of capacity (0..1)
        battery_discharge_percent: float = 0.25,    # max charge/discharge power per hour as % of capacity
        battery_min_soc_percent: float = 0.10,
        battery_max_soc_percent: float = 0.90,
    ):
        self.name = name
        self.n_units = int(n_units)


        self.timestamp_tracker = []  # for tracking the timestamps of each step
        # annual demands
        self.annual_elec_kWh_per_unit = float(annual_elec_kWh_per_unit)
        self.annual_heat_kWh_per_unit = float(annual_heat_kWh_per_unit)



        # profiles
        self.nominal_elec_profile = nominal_elec_profile
        self.heating_profile = heating_profile

        self.actual_elec_profile = []  # for KPI tracking
        self.actual_heat_profile = []  # for KPI tracking

        # PV
        self.pv_kWp_per_unit = float(pv_kWp_per_unit)
        self.pv_performance_ratio = float(pv_performance_ratio)
        #KPI
        self.pv_generation_kWh = []

        # Heat pump (HVAC)
        self.hvac_max_elec_kW_per_unit = float(hvac_max_elec_kW_per_unit)
            # KPI
        self.hvac_electricity_consumption_kWh = []
        self.hvac_heat_production_kWh = []
        self.hvac_cop_carnot = []
        self.hvac_cop_actual = []        
        self.T_source = [] #IN C, not Kelvin. T_ambient or river for Industrial HVAC
        


        # Resistive heater
        self.eheater_max_kW_per_unit = float(eheater_max_kW_per_unit)
            #KPI    
        self.eheater_consumption_kWh = []
        self.eheater_production_kWh = []

        # Gas boiler  #DONE gas_boiler_gas_consumption_kWh
        self.gas_boiler_max_kW_per_unit = float(gas_boiler_max_kW_per_unit)
        self.gas_boiler_efficiency = float(gas_boiler_efficiency)
            # KPI 
        self.gas_boiler_gas_consumption_kWh = []
        self.gas_boiler_heat_production_kWh = []
        # 
        # Micro-CHP DONE 
        self.chp_elec_cap_kW_per_unit = float(chp_elec_cap_kW_per_unit)
        self.chp_eta_elec = float(chp_eta_elec)
        self.chp_eta_heat = float(chp_eta_heat)
            #KPI
        self.chp_gas_consumption_kWh = []
        self.chp_heat_production_kWh = []
        self.chp_electricity_production_kWh = []
        # Battery
        self.battery_capacity_kWh = float(battery_capacity_kWh)
        self.battery_discharge_percent = float(battery_discharge_percent)
        self.battery_min_soc_percent = float(battery_min_soc_percent)
        self.battery_max_soc_percent = float(battery_max_soc_percent)
            # KPI
        self.battery_soc_history_kWh = []
        self.battery_soc_history_percent = []


        # internal battery state
        cap = self.battery_capacity_kWh * self.n_units
        self.battery_soc_kWh = cap * float(initial_battery_soc_percent) if cap > 0 else 0.0




    # --------- STEP ----------
    def step(self, t: pd.Timestamp, signals: pd.Series | Dict[str, float], dt_h: float) -> Dict[str, float]:
        # --- inputs ---
        p_kWh = _get(signals, "P[kWh]", 0.0)        # we are using the PV ready data from pvgis ... just scaling with the kWp and we good
        tout = _get(signals, "ambient_temp_C", 15.0)      # °C

        # --- base demands from profiles (kW as average over the step) ---
        # profiles normalized to 1: per-step energy = annual * weight; power = energy/dt
        conv = 1.0 / max(dt_h, 1e-9)
        base_elec_kW = self.n_units * self.annual_elec_kWh_per_unit * float(self.nominal_elec_profile.loc[t]) * conv
        heat_need_kW = self.n_units * self.annual_heat_kWh_per_unit * float(self.heating_profile.loc[t]) * conv

        self.actual_elec_profile.append(base_elec_kW  )
        self.actual_heat_profile.append(heat_need_kW  )
        # --- PV generation (kW) ---
        pv_kW = self._pv_power(p_kWh)



        self.timestamp_tracker.append(t)  # track the timestamp of this step
        # --- Flexible device dispatch order ---
        def device_dispatch(heat_need_kW, tout, order=None):
            # Default order if not provided
            if order is None:
                order = [
                    "chp",      # micro-CHP
                    "hvac",     # heat pump
                    "eheater",  # resistive heater
                    "gas_boiler" # gas boiler
                ]
            # State variables
            state = {
            "heat_remaining": heat_need_kW,
            "chp_elec_kW": 0.0, "chp_heat_kW": 0.0, "chp_gas_kW": 0.0,
            "hvac_elec_kW": 0.0, "hvac_heat_kW": 0.0,
            "eheater_elec_kW": 0.0, "eheater_heat_kW": 0.0,
            "gas_heat_kW": 0.0, "gas_input_kW": 0.0,
            }
            for device in order:
                if device == "chp":
                    chp = self._micro_chp(state["heat_remaining"])
                    state["chp_elec_kW"] = chp["chp_elec_kW"]
                    state["chp_heat_kW"] = chp["chp_heat_kW"]
                    state["chp_gas_kW"] = chp["chp_gas_kW"]
                    state["heat_remaining"] = max(0.0, state["heat_remaining"] - chp["chp_heat_kW"])
                elif device == "hvac":
                    hvac = self._hvac_dispatch(tout, state["heat_remaining"])
                    state["hvac_elec_kW"] = hvac["hvac_elec_kW"]
                    state["hvac_heat_kW"] = hvac["hvac_heat_kW"]
                    state["heat_remaining"] = max(0.0, state["heat_remaining"] - hvac["hvac_heat_kW"])
                elif device == "eheater":
                    eh = self._electric_heater(state["heat_remaining"])
                    state["eheater_elec_kW"] = eh["eheater_elec_kW"]
                    state["eheater_heat_kW"] = eh["eheater_heat_kW"]
                    state["heat_remaining"] = max(0.0, state["heat_remaining"] - eh["eheater_heat_kW"])
                elif device == "gas_boiler":
                    gb = self._gas_boiler(state["heat_remaining"])
                    state["gas_heat_kW"] = gb["gas_heat_kW"]
                    state["gas_input_kW"] = gb["gas_input_kW"]
                    state["heat_remaining"] = max(0.0, state["heat_remaining"] - gb["gas_heat_kW"])
            
            return state

        # Call the dispatch function with desired order (default order as before)
        dispatch = device_dispatch(heat_need_kW, tout)
        chp_elec_kW = dispatch["chp_elec_kW"]           # output electricity chp
        chp_heat_kW = dispatch["chp_heat_kW"]           # output heat chp
        chp_gas_kW  = dispatch["chp_gas_kW"]            # input fuel chp
        hvac_elec_kW = dispatch["hvac_elec_kW"]         # electric input hvac
        hvac_heat_kW = dispatch["hvac_heat_kW"]         # heat output hvac
        eheater_elec_kW = dispatch["eheater_elec_kW"]   # electric input e-heater
        eheater_heat_kW = dispatch["eheater_heat_kW"]   # heat output e-heater
        gas_heat_kW = dispatch["gas_heat_kW"]           # heat output gas boiler
        gas_input_kW = dispatch["gas_input_kW"]         # input fuel gas boiler 

        # --- Energy balance before battery ---
        total_gas_input_kW = chp_gas_kW  + gas_input_kW  # total gas input
        
        elec_demand_kW = base_elec_kW + hvac_elec_kW + eheater_elec_kW
        elec_supply_kW = pv_kW + chp_elec_kW
        net_elec_kW = elec_demand_kW - elec_supply_kW  # + = import, - = export

        # --- Battery (optional) ---
        batt = self._battery_step(net_elec_kW, dt_h)
        grid_kW = batt["grid_kW"]
        batt_chg_kW = batt["battery_charge_kW"]
        batt_dis_kW = batt["battery_discharge_kW"]
        self.battery_soc_kWh = batt["battery_soc_kWh"]

        # --- PV self-consumption (simple attribution) ---
        # PV used on site = min(PV, demand + battery charging)
        selfconsumed_kW = min(pv_kW, elec_demand_kW + max(batt_chg_kW, 0.0)) #TODO ADD CHP TO THE SELF CONSUMPTION!!!! BUT its a nice Idea to have "Green" vs "Grey" self consumption
        exported_kW = max(0.0, -grid_kW)  # net export to grid

        
        return {
            # stored by the Simulation:
            "electricity_needed_kW": elec_demand_kW,
            "heat_needed_kW":        heat_need_kW,
            "gas_input_kW":          total_gas_input_kW,
            "grid_kW":               grid_kW,
            "selfconsumed_kW":       selfconsumed_kW,
            "exported_kW":           exported_kW,
            # extras you may want for debugging (ignored by the Simulation writer):
            "pv_kW": pv_kW, "chp_elec_kW": chp_elec_kW, "chp_heat_kW": chp_heat_kW,
            "hvac_elec_kW": hvac_elec_kW, "hvac_heat_kW": hvac_heat_kW,
            "eheater_elec_kW": eheater_elec_kW, "eheater_heat_kW": eheater_heat_kW,
        }

    # --------- DEVICES (very simple models) ---------
    def _pv_power(self, signals: pd.Series | dict) -> float:
        """
        Compute PV power (kW) given either:
        - 'P[kWh]'  = specific energy (kWh/kWp/step), or
        - 'P[W]'    = specific power (W/kWp).
        Falls back to irradiance if neither is present.
        """
        kWp_total = self.pv_kWp_per_unit * self.n_units
        if kWp_total <= 0:
            self.pv_generation_kWh.append(0.0)
            return 0.0

    # Prefer energy per step if provided

        p_kwh = max(0.0, float(signals))
        self.pv_generation_kWh.append(p_kwh * kWp_total)
        return (kWp_total * p_kwh) 

        # # Next: specific power (W/kWp)
        # if "P[W]" in signals:
        #     p_W = max(0.0, float(signals["P[W]"]))
        #     return kWp_total * p_W / 1000.0                          # kW, no PR

        # # Fallback: irradiance path with PR
        # irr = _get(signals, "irradiance_Wm2", 0.0)
        # return max(0.0, kWp_total * (irr / 1000.0) * self.pv_performance_ratio)


    def _micro_chp(self, heat_need_kW: float) -> Dict[str, float]:
        if self.chp_elec_cap_kW_per_unit <= 0 or (self.chp_eta_elec <= 0) or (self.chp_eta_heat <= 0):
            
            self.chp_gas_consumption_kWh.append(0.0)
            self.chp_heat_production_kWh.append(0.0)
            self.chp_electricity_production_kWh.append(0.0)
            
            return {"chp_elec_kW": 0.0, "chp_heat_kW": 0.0, "chp_gas_kW": 0.0}

        elec_cap = self.chp_elec_cap_kW_per_unit * self.n_units
        # heat produced per kW_elec = eta_heat/eta_elec * kW_elec
        heat_per_kw_elec = self.chp_eta_heat / max(self.chp_eta_elec, 1e-9)
        # run enough electric power to cover as much heat as possible (but <= cap)
        needed_elec_for_heat = heat_need_kW / max(heat_per_kw_elec, 1e-9)
        chp_elec = min(elec_cap, needed_elec_for_heat)
        chp_heat = chp_elec * heat_per_kw_elec
        chp_gas  = chp_elec / max(self.chp_eta_elec, 1e-9)  # input fuel power
        
        self.chp_gas_consumption_kWh.append(chp_gas)
        self.chp_heat_production_kWh.append(chp_heat)
        self.chp_electricity_production_kWh.append(chp_elec)
        
        return {"chp_elec_kW": chp_elec, "chp_heat_kW": chp_heat, "chp_gas_kW": chp_gas}

    def _hvac_dispatch(self, ambient_temp_C: float, heat_need_kW: float) -> Dict[str, float]:
        """
        Heat pump dispatch using a simple Carnot COP with an efficiency factor.
        Pass ambient air temperature for ASHPs as `ambient_temp_C`.
        For water-source plants you can either:
        - set self.hvac_source_temp_C_override to a float (or a per-step value in your step()), OR
        - call this method with the river temp as ambient_temp_C.

        Returns:
            dict: {
            'hvac_elec_kW', 'hvac_heat_kW', 'cop',
            'cop_carnot', 'T_source_C', 'T_sink_C'
            }
        """
        max_in = self.hvac_max_elec_kW_per_unit * self.n_units
        if max_in <= 0 or heat_need_kW <= 0:
            self.hvac_electricity_consumption_kWh.append(0.0)
            self.hvac_heat_production_kWh.append(0.0)
            self.hvac_cop_carnot.append(float("nan"))
            self.hvac_cop_actual.append(float("nan"))
            self.T_source.append(float("nan"))
            
            return {"hvac_elec_kW": 0.0, "hvac_heat_kW": 0.0, "cop": float("nan"),
                    "cop_carnot": float("nan"), "T_source_C": float("nan"), "T_sink_C": float("nan")}

        # --------- Source & sink temperatures (°C) ---------
        # HACK: If an override exists (e.g., river water temp), use it; else use ambient air.
        source_C = getattr(self, "hvac_source_temp_C_override", None)   #<-- USER OVERRIDE IMPORTANT 
        if source_C is None:
            source_C = float(ambient_temp_C)

        # Delivery (sink) setpoint. Households often ~35 °C (LT heating),
        # industrial could be ~60–80 °C. Tunable per instance.
        sink_C = float(getattr(self, "hvac_sink_temp_C", 35.0))

        # Approach temperatures (K) to avoid ideal heat exchanger assumptions.
        dT_src_K = float(getattr(self, "hvac_source_approach_K", 3.0))  # source side temp lift (extraction)
        dT_snk_K = float(getattr(self, "hvac_sink_approach_K",   5.0))  # sink side temp approach (delivery)

        # Convert to Kelvin and apply approaches
        T_cold_K = (source_C - dT_src_K) + 273.15
        T_hot_K  = (sink_C   + dT_snk_K) + 273.15

        # Guard against non-physical or near-equal temps
        dT_K = max(1e-3, T_hot_K - max(T_cold_K, 1.0))
        cop_carnot = T_hot_K / dT_K

        # Real-world efficiency relative to Carnot (0–1). Typical 0.4–0.6.
        eta_carnot = float(getattr(self, "hvac_eta_carnot", 0.5))
        cop = eta_carnot * cop_carnot

        # Keep COP in a reasonable range
        cop_min = float(getattr(self, "hvac_cop_min", 1.0)) # MAYBE 2 OR 1.5? TODO: CHECK
        cop_max = float(getattr(self, "hvac_cop_max", 10.0))
        cop = float(np.clip(cop, cop_min, cop_max))

        # Required electric input to meet heat_need_kW (subject to input cap)
        needed_in_kW = heat_need_kW / max(cop, 1e-9)
        hvac_elec_kW = min(max_in, needed_in_kW)
        hvac_heat_kW = hvac_elec_kW * cop


        self.hvac_electricity_consumption_kWh.append(hvac_elec_kW)
        self.hvac_heat_production_kWh.append(hvac_heat_kW)
        self.hvac_cop_carnot.append(float(cop_carnot))
        self.hvac_cop_actual.append(float(cop))
        self.T_source.append(float(source_C))
        
        return {
            "hvac_elec_kW": hvac_elec_kW,
            "hvac_heat_kW": hvac_heat_kW,
            "cop": cop,
            # extras that are handy to log/plot:
            "cop_carnot": float(cop_carnot),
            "T_source_C": float(source_C),
            "T_sink_C": float(sink_C),
        }

    def _electric_heater(self, heat_remaining_kW: float) -> Dict[str, float]:
        cap = self.eheater_max_kW_per_unit * self.n_units
        if cap <= 0 or heat_remaining_kW <= 0:
            self.eheater_consumption_kWh.append(0.0)
            self.eheater_production_kWh.append(0.0)
            return {"eheater_elec_kW": 0.0, "eheater_heat_kW": 0.0}
        elec = min(cap, heat_remaining_kW)
        self.eheater_consumption_kWh.append(elec)
        self.eheater_production_kWh.append(elec)
        return {"eheater_elec_kW": elec, "eheater_heat_kW": elec}  # 1:1

    def _gas_boiler(self, heat_remaining_kW: float) -> Dict[str, float]:   #TODO: GASBOILER, mostly doneADD THE GAS CONSUMED IN m3 or whatever units 
        """
        Dispatch the gas boiler, respecting max thermal output per unit.
        Returns:
            dict: {"gas_heat_kW", "gas_input_kW"}
        """
        if heat_remaining_kW <= 0:
            self.gas_boiler_gas_consumption_kWh.append(0.0)
            self.gas_boiler_heat_production_kWh.append(0.0)
            return {"gas_heat_kW": 0.0, "gas_input_kW": 0.0}

        # Total max thermal output (kW) = per-unit cap × number of units
        cap = self.gas_boiler_max_kW_per_unit * self.n_units
        if cap <= 0:
            self.gas_boiler_gas_consumption_kWh.append(0.0)
            self.gas_boiler_heat_production_kWh.append(0.0)
            
            return {"gas_heat_kW": 0.0, "gas_input_kW": 0.0}

        # Thermal output is limited by both demand and cap
        gas_heat = min(heat_remaining_kW, cap)

        # Gas input (kW fuel) accounts for efficiency
        eff = max(self.gas_boiler_efficiency, 1e-6)
        gas_in = gas_heat / eff
        
        self.gas_boiler_gas_consumption_kWh.append(gas_in)
        self.gas_boiler_heat_production_kWh.append(gas_heat)
        return {"gas_heat_kW": gas_heat, "gas_input_kW": gas_in}

    def _battery_step(self, net_elec_kW: float, dt_h: float) -> Dict[str, float]:

        cap = self.battery_capacity_kWh * self.n_units
        if cap <= 0:
            
            self.battery_soc_history_kWh.append(0.0)
            self.battery_soc_history_percent.append(0.0)
            
            return {
                "grid_kW": net_elec_kW,
                "battery_charge_kW": 0.0,
                "battery_discharge_kW": 0.0,
                "battery_soc_kWh": 0.0,
            }

        # power limit (kW)
        p_lim = max(0.0, self.battery_discharge_percent) * cap  # kWh/h = kW
        soc_min = cap * max(0.0, min(1.0, self.battery_min_soc_percent))
        soc_max = cap * max(0.0, min(1.0, self.battery_max_soc_percent))

        soc = float(self.battery_soc_kWh)

        charge_kW = 0.0
        discharge_kW = 0.0

        if net_elec_kW > 0:  # importing → try discharging
            # max energy we can extract this step:
            max_discharge_energy = max(0.0, soc - soc_min)
            max_discharge_kW = min(p_lim, max_discharge_energy / max(dt_h, 1e-9))
            discharge_kW = min(net_elec_kW, max_discharge_kW)
            soc -= discharge_kW * dt_h
            grid_kW = net_elec_kW - discharge_kW

        else:  # exporting → try charging
            surplus = -net_elec_kW
            headroom = max(0.0, soc_max - soc)
            max_charge_kW = min(p_lim, headroom / max(dt_h, 1e-9))
            charge_kW = min(surplus, max_charge_kW)
            soc += charge_kW * dt_h
            grid_kW = -(surplus - charge_kW)  # still negative if remaining export

        self.battery_soc_history_kWh.append(soc)
        self.battery_soc_history_percent.append(soc / cap if cap > 0 else 0.0)
        return {
            "grid_kW": grid_kW,
            "battery_charge_kW": charge_kW,
            "battery_discharge_kW": discharge_kW,
            "battery_soc_kWh": soc,
        }
    
    def generate_results(self) -> pd.DataFrame:
        """
        Generate a DataFrame with time series of key metrics and KPIs.
        """
        data = {
            "timestamp": self.timestamp_tracker,
            "electricity_needed_kW": self.actual_elec_profile,
            "heat_needed_kW": self.actual_heat_profile,
            "hvac_electricity_consumption_kWh": self.hvac_electricity_consumption_kWh,
            "hvac_heat_production_kWh": self.hvac_heat_production_kWh,
            "hvac_cop_carnot": self.hvac_cop_carnot,
            "hvac_cop_actual": self.hvac_cop_actual,
            "T_source_C": self.T_source,
            "eheater_consumption_kWh": self.eheater_consumption_kWh,
            "eheater_production_kWh": self.eheater_production_kWh,
            "gas_boiler_gas_consumption_kWh": self.gas_boiler_gas_consumption_kWh,
            "gas_boiler_heat_production_kWh": self.gas_boiler_heat_production_kWh,
            "chp_gas_consumption_kWh": self.chp_gas_consumption_kWh,
            "chp_heat_production_kWh": self.chp_heat_production_kWh,
            "chp_electricity_production_kWh": self.chp_electricity_production_kWh,
            "pv_generation_kWh": self.pv_generation_kWh,
            "battery_soc_history_kWh": self.battery_soc_history_kWh,
            "battery_soc_history_percent": self.battery_soc_history_percent,
        }
        return data


    def pre_inspect(
        self,
        df_external_data: pd.DataFrame | None = None,
        start: pd.Timestamp | str | None = None,
        end: pd.Timestamp | str | None = None,
        window: str = "7D",
        show: bool = True,
        return_fig: bool = True,
    ):
        """
        Quick pre-run inspection: prints a summary and plots
        - per-unit & fleet electric/heat (kW) from normalized profiles
        - ambient temperature (°C) if available
        - PV potential (kW) if irradiance is available and PV is sized

        Args:
            df_external_data: optional dataframe aligned (or alignable) to the subject index.
                            Columns used if present: 'ambient_temp_C', 'irradiance_Wm2'.
            start, end: optional window bounds; if not provided, plots the first `window` (e.g. "7D")
            window: pandas offset string (used only if start/end not specified)
            show: display the figure (Jupyter/Notebook)
            return_fig: return the plotly figure

        Returns:
            dict with 'summary' and optionally 'fig'
        """
        # ---------- helpers ----------
        def _dt_series(index: pd.DatetimeIndex) -> pd.Series:
            if len(index) <= 1:
                # fall back to 1 hour to avoid div-by-zero in degenerate cases
                return pd.Series([1.0], index=index)
            d = index.to_series().diff().dt.total_seconds().div(3600.0)
            # first step ~ second-first
            d.iloc[0] = d.iloc[1]
            return d

        def _norm_check(series: pd.Series, name: str) -> float:
            s = float(series.sum())
            if abs(s - 1.0) > 1e-3:
                print(f"⚠️  {self.name}: {name} sum = {s:.6f} (not ~1.0) — normalize your profile.")
            return s

        # ---------- index & window ----------
        # Use the electricity profile index as the canonical index
        idx = self.nominal_elec_profile.index
        elec_prof = self.nominal_elec_profile.reindex(idx).fillna(0.0)
        heat_prof = self.heating_profile.reindex(idx).fillna(0.0)

        # Choose plotting window
        if start is None and end is None:
            start = idx.min()
            end = start + pd.to_timedelta(window)
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        win = slice(start, end)
        win_idx = idx[(idx >= start) & (idx <= end)]

        # ---------- per-step dt and power series ----------
        dt_h = _dt_series(idx)
        # energy per step (kWh per unit) = annual_kWh_per_unit * normalized_weight
        elec_kWh_unit = self.annual_elec_kWh_per_unit * elec_prof
        heat_kWh_unit = self.annual_heat_kWh_per_unit * heat_prof
        # convert to average power over the step (kW) = kWh / h
        elec_kW_unit = elec_kWh_unit.div(dt_h).reindex(idx)
        heat_kW_unit = heat_kWh_unit.div(dt_h).reindex(idx)
        # fleet totals
        elec_kW_fleet = elec_kW_unit * self.n_units
        heat_kW_fleet = heat_kW_unit * self.n_units

        # ---------- external overlays ----------
        amb = irr = pv_potential_fleet_kW = None
        if df_external_data is not None:
            # align and slice
            ext = df_external_data.reindex(idx)
            if "ambient_temp_C" in ext.columns:
                amb = ext["ambient_temp_C"].loc[win].astype(float)
            if "irradiance_Wm2" in ext.columns:
                irr = ext["irradiance_Wm2"].loc[win].astype(float)
                # PV potential on the fleet with current PR
                kWp_total = self.pv_kWp_per_unit * self.n_units
                if kWp_total > 0:
                    pv_potential_fleet_kW = (kWp_total * self.pv_performance_ratio * (irr / 1000.0)).rename("PV potential (fleet) kW")



        w_elec_unit = float((elec_kW_unit.loc[win] * dt_h.loc[win]).sum())
        w_heat_unit = float((heat_kW_unit.loc[win] * dt_h.loc[win]).sum())

                # ---------- print summary ----------
        elec_sum = _norm_check(elec_prof, "nominal_elec_profile")
        heat_sum = _norm_check(heat_prof, "heating_profile")

        # max values
        max_elec_unit = float(elec_kW_unit.max())
        max_heat_unit = float(heat_kW_unit.max())
        max_elec_fleet = float(elec_kW_fleet.max())
        max_heat_fleet = float(heat_kW_fleet.max())

        print(f"\n⚡️🔥 — PRE-INSPECT: {self.name} — ⚡️🔥")
        print(f"👥 Units: {self.n_units}")
        print(f"🗓️ Index: {idx.min()} → {idx.max()}  | steps={len(idx)}  | median dt={dt_h.median():.3f} h")
        print(f"📊 Annual per unit: Elec={self.annual_elec_kWh_per_unit:.0f} kWh, Heat={self.annual_heat_kWh_per_unit:.0f} kWh")
        print(f"📊 Annual fleet:    Elec={self.annual_elec_kWh_per_unit*self.n_units:.0f} kWh, "
              f"Heat={self.annual_heat_kWh_per_unit*self.n_units:.0f} kWh")
        print(f"🪟 Window {start} → {end}: per-unit energy  "
              f"Elec={w_elec_unit:.1f} kWh, Heat={w_heat_unit:.1f} kWh")

        print("\n🚀 Maximum demand (kW):")
        print(f"   • Per unit:   ⚡ Elec={max_elec_unit:.2f} kW | 🔥 Heat={max_heat_unit:.2f} kW")
        print(f"   • Fleet:      ⚡ Elec={max_elec_fleet:.2f} kW | 🔥 Heat={max_heat_fleet:.2f} kW")


        # ---------- plot ----------
        fig = None
        try:
            from plotly.subplots import make_subplots
            import plotly.graph_objects as go

            fig = make_subplots(specs=[[{"secondary_y": True}]])
            # per-unit
            fig.add_trace(go.Scatter(x=win_idx, y=elec_kW_unit.loc[win_idx], name="Elec / unit (kW)"))
            fig.add_trace(go.Scatter(x=win_idx, y=heat_kW_unit.loc[win_idx], name="Heat / unit (kW)"))
            # fleet (fainter)
            fig.add_trace(go.Scatter(x=win_idx, y=elec_kW_fleet.loc[win_idx], name="Elec / fleet (kW)", opacity=0.35))
            fig.add_trace(go.Scatter(x=win_idx, y=heat_kW_fleet.loc[win_idx], name="Heat / fleet (kW)", opacity=0.35))

            # ambient temp on secondary axis
            if amb is not None:
                fig.add_trace(go.Scatter(x=amb.index, y=amb, name="Ambient (°C)"), secondary_y=True)
            # PV potential (fleet)
            if pv_potential_fleet_kW is not None:
                fig.add_trace(go.Scatter(x=pv_potential_fleet_kW.index, y=pv_potential_fleet_kW, name="PV potential fleet (kW)"))

            fig.update_layout(
                title=f"Pre-inspect — {self.name}  |  window: {start} → {end}",
                hovermode="x unified",
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
                margin=dict(l=50, r=50, t=60, b=40),
            )
            fig.update_xaxes(title_text="time")
            fig.update_yaxes(title_text="kW (demand/generation)", secondary_y=False)
            fig.update_yaxes(title_text="°C (ambient)", secondary_y=True)

            if show:
                fig.update_layout(width=1100, height=600)
                fig.show()
        except Exception as e:
            print(f"(Plotly unavailable? Skipping figure. Details: {e})")

        out = {"summary": {
            "name": self.name,
            "n_units": self.n_units,
            "index_start": idx.min(),
            "index_end": idx.max(),
            "steps": len(idx),
            "median_dt_h": float(dt_h.median()),
            "annual_per_unit_elec_kWh": float(self.annual_elec_kWh_per_unit),
            "annual_per_unit_heat_kWh": float(self.annual_heat_kWh_per_unit),
            "annual_fleet_elec_kWh": float(self.annual_elec_kWh_per_unit * self.n_units),
            "annual_fleet_heat_kWh": float(self.annual_heat_kWh_per_unit * self.n_units),
            "profile_sum_elec": elec_sum,
            "profile_sum_heat": heat_sum,
            "window_per_unit_elec_kWh": w_elec_unit,
            "window_per_unit_heat_kWh": w_heat_unit,
        }}
        if return_fig:
            out["fig"] = fig
        return out




In [ ]:
# results_df = subject.generateResults()

In [142]:

class Simulation:
    """Minimal loop-based engine (teaching version)."""

    def __init__(self, subjects: List[Any] | None = None, verbose: bool = True):
        self.subjects = subjects if subjects is not None else []
        self.verbose = verbose

    def run(self, df_external_data: pd.DataFrame) -> pd.DataFrame:
        if self.verbose:
            print("Starting minimal Simulation...\n")

        t0 = time.time()
        df = df_external_data.copy()

        # Ensure DatetimeIndex
        if not isinstance(df.index, pd.DatetimeIndex):
            if "datetime" in df.columns:
                df["datetime"] = pd.to_datetime(df["datetime"])
                df = df.set_index("datetime")
            else:
                raise ValueError("Provide a DatetimeIndex or a 'datetime' column.")

        # Make sure sorted
        df = df.sort_index()

        # dt_h per row (assume constant step if single row)
        idx = df.index
        if len(idx) == 1:
            dt_h = pd.Series([0.0], index=idx)
        else:
            step0 = (idx[1] - idx[0]).total_seconds() / 3600.0
            deltas = idx.to_series().diff().dt.total_seconds().div(3600.0)
            deltas.iloc[0] = step0
            dt_h = deltas
        df["dt_h"] = dt_h

        # Pre-create result columns
        for subj in self.subjects:
            prefix = f"{subj.name}_"
            for out_key in METRICS.keys():
                df[prefix + out_key] = 0.0

        # Loop
        try:
            from tqdm import tqdm  # optional progress bar
            iterator = tqdm(df.itertuples(index=True), total=len(df), desc="Simulating")
        except Exception:
            iterator = df.itertuples(index=True)

        for row in iterator:
            t = row.Index
            row_series = df.loc[t]  # signals for this step
            step_dt = float(row_series["dt_h"])

            for subj in self.subjects:
                try:
                    raw = subj.step(t, row_series, step_dt)
                except Exception as e:
                    print(f"⚠️ Error in {subj.name}.step at {t}: {e}")
                    raw = {}  # fallback

                # write the subset we persist
                for out_name, src in METRICS.items():
                    try:
                        df.at[t, f"{subj.name}_{out_name}"] = float(raw.get(src, 0.0))
                    except Exception as e:
                        print(f"⚠️ Error writing {subj.name}_{out_name} at {t}: {e}")
                        df.at[t, f"{subj.name}_{out_name}"] = 0.0

        if self.verbose:
            st, et = df.index.min(), df.index.max()
            dur_h = float(df["dt_h"].sum())
            print(f"\nWindow: {st} → {et}  (duration {dur_h:.2f} h, steps={len(df)})")

        t1 = time.time()
        if self.verbose:
            print(f"✅ Simulation finished in {t1 - t0:.2f} s.\n")

        return df


In [154]:
df_elec.head(1)

,household,general_commerce,commerce_office_day,commerce_evening,commerce_continuous,shop_barber,bakery,weekend_operation,farming_general,farming_dairy,farming_other
datetime,,,,,,,,,,,
2021-01-01,0.000017,0.000016,0.000006,0.000019,0.000024,0.000015,0.000012,0.000016,0.000018,0.000017,0.000019


In [155]:
df_heat.head(1)

,single_family_house,multi_family_house,metal_automotive,retail_wholesale,authorities_finance_insurance,other_services,restaurants,accommodation,laundries_drycleaning,horticulture,bakery,paper_printing,household_like_business,commerce_services_total
datetime,,,,,,,,,,,,,,
2021-01-01,0.000032,0.00003,0.000058,0.000046,0.000049,0.00007,0.000048,0.000037,0.000005,0.000083,0.000056,0.00007,0.000031,0.000041


In [ ]:
#



#
elec_household = df_elec["household"]
heat_sfh       = df_heat["single_family_house"]
heat_mfh       = df_heat["multi_family_house"]

# ----- reasonable (editable) annual energy assumptions -----
# per HOME (kWh/year)

ELEC_PER_FLAT     = 3300
HEAT_PER_FLAT     = 12000
FLATS_PER_BUILDING = 20  # change to your stock → affects “per building” totals



# convert to "per unit" **building** totals where a unit = one building
ELEC_PER_FLATS_BUILDING = ELEC_PER_FLAT * FLATS_PER_BUILDING
HEAT_PER_FLATS_BUILDING = HEAT_PER_FLAT * FLATS_PER_BUILDING

# ----- subjects -----
flats = Consumer(
    name="flats",
    n_units=1,  # 1 building
    annual_elec_kWh_per_unit=ELEC_PER_FLATS_BUILDING,
    annual_heat_kWh_per_unit=HEAT_PER_FLATS_BUILDING,
    nominal_elec_profile=df_elec["household"],
    heating_profile=df_heat["single_family_house"],

    # PV
    pv_kWp_per_unit=100.0,   #NOMINAL
    pv_performance_ratio=0.85,
    # Heat pump (HVAC)
    hvac_max_elec_kW_per_unit=1.0, #NOMINAL
    # Resistive heater
    eheater_max_kW_per_unit=1.0, #NOMINAL
    # Gas boiler
    gas_boiler_max_kW_per_unit=1.0, #NOMINAL
    gas_boiler_efficiency=0.90,
    # Micro-CHP (optional; heat-led)
    chp_elec_cap_kW_per_unit=1.0, #NOMINAL
    chp_eta_elec=0.30,
    chp_eta_heat=0.50,
    # Battery (optional)
    battery_capacity_kWh         = 10.0, #NOMINAL
    initial_battery_soc_percent  = 0.9,   # fraction of capacity (0..1)
    battery_discharge_percent     = 0.25,    # max charge/discharge power per hour as % of capacity
    battery_min_soc_percent       = 0.10,
    battery_max_soc_percent       = 0.90,
    )

# After you build the subject
sfh = flats  # e.g., from your earlier setup
sfh.pre_inspect(df_external_data["2021":"2023"], start="2023-01-01", end="2023-02-14",return_fig=False)


In [ ]:
# CHRIST

BS = Consumer(
    name="business_shop",
    n_units=3,
    annual_elec_kWh_per_unit=27000,
    annual_heat_kWh_per_unit=12000,
    nominal_elec_profile=df_elec["shop_barber"],
    heating_profile=df_heat["other_services"],
)

BB = Consumer(
    name="business_bakery",
    n_units=1,
    annual_elec_kWh_per_unit=8200,
    annual_heat_kWh_per_unit=6600,
    nominal_elec_profile=df_elec["bakery"],
    heating_profile=df_heat["bakery"],
)

BW = Consumer(
    name="business_weekend",
    n_units=1,
    annual_elec_kWh_per_unit=100000,
    annual_heat_kWh_per_unit=80000,
    nominal_elec_profile=df_elec["weekend_operation"],
    heating_profile=df_heat["restaurants"],
)

FA = Consumer(
    name="farmA",
    n_units=5,
    annual_elec_kWh_per_unit=36320,
    annual_heat_kWh_per_unit=7040,
    nominal_elec_profile=df_elec["farming_dairy"],
    heating_profile=df_heat["horticulture"],
)

FB = Consumer(
    name="farmB",
    n_units=1,
    annual_elec_kWh_per_unit=13700,
    annual_heat_kWh_per_unit=7040,
    nominal_elec_profile=df_elec["farming_other"],
    heating_profile=df_heat["horticulture"],
)

In [ ]:
# df_external_data["ambient_temp_C"]... where index.month == [12, 1, 2] .... -3 .. else +5 ....

In [ ]:
# ----- run -----
subjects = [flats]
sim = Simulation(subjects=subjects, verbose=True)
results = sim.run(df_external_data["2021":"2023"])  # first 4 days of 2023 for quick test

Starting minimal Simulation...



Simulating: 100%|██████████| 35040/35040 [00:09<00:00, 3685.97it/s]

⚠️ Error in flats.step at 2023-12-31 23:15:00: Timestamp('2023-12-31 23:15:00')
⚠️ Error in flats.step at 2023-12-31 23:30:00: Timestamp('2023-12-31 23:30:00')
⚠️ Error in flats.step at 2023-12-31 23:45:00: Timestamp('2023-12-31 23:45:00')

Window: 2023-01-01 00:00:00 → 2023-12-31 23:45:00  (duration 8760.00 h, steps=35040)
✅ Simulation finished in 9.52 s.



In [ ]:
#
df_results_flats = pd.DataFrame(flats.generate_results())
df_results_flats["timestamp"] = pd.to_datetime(df_results_flats["timestamp"])
df_results_flats.set_index("timestamp", inplace=True)
df_results_flats.head(3)



,electricity_needed_kW,heat_needed_kW,hvac_electricity_consumption_kWh,hvac_heat_production_kWh,hvac_cop_carnot,hvac_cop_actual,T_source_C,eheater_consumption_kWh,eheater_production_kWh,gas_boiler_gas_consumption_kWh,gas_boiler_heat_production_kWh,chp_gas_consumption_kWh,chp_heat_production_kWh,chp_electricity_production_kWh,pv_generation_kWh,battery_soc_history_kWh,battery_soc_history_percent
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,5.779301,6.565338,1.0,4.509973,9.019947,4.509973,8.2825,0.388698,0.388698,0.0,0.0,3.333333,1.666667,1.0,0.0,8.375,0.8375
2023-01-01 00:15:00,5.361584,6.565338,1.0,4.501869,9.003738,4.501869,8.2200,0.396802,0.396802,0.0,0.0,3.333333,1.666667,1.0,0.0,7.750,0.7750
2023-01-01 00:30:00,4.957086,6.565338,1.0,4.499605,8.999210,4.499605,8.2025,0.399066,0.399066,0.0,0.0,3.333333,1.666667,1.0,0.0,7.125,0.7125


In [ ]:
# df_results_flats["elec_price_EUR_per_MWh"] = df_external_data["elec_price_EUR_per_MWh"]
df_results_flats

,electricity_needed_kW,heat_needed_kW,hvac_electricity_consumption_kWh,hvac_heat_production_kWh,hvac_cop_carnot,hvac_cop_actual,T_source_C,eheater_consumption_kWh,eheater_production_kWh,gas_boiler_gas_consumption_kWh,gas_boiler_heat_production_kWh,chp_gas_consumption_kWh,chp_heat_production_kWh,chp_electricity_production_kWh,pv_generation_kWh,battery_soc_history_kWh,battery_soc_history_percent,elec_price_EUR_per_MWh
timestamp,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,5.779301,6.565338,1.0,4.509973,9.019947,4.509973,8.2825,0.388698,0.388698,0.000000,0.000000,3.333333,1.666667,1.0,0.0,8.375,0.8375,75.010
2023-01-01 00:15:00,5.361584,6.565338,1.0,4.501869,9.003738,4.501869,8.2200,0.396802,0.396802,0.000000,0.000000,3.333333,1.666667,1.0,0.0,7.750,0.7750,-29.950
2023-01-01 00:30:00,4.957086,6.565338,1.0,4.499605,8.999210,4.499605,8.2025,0.399066,0.399066,0.000000,0.000000,3.333333,1.666667,1.0,0.0,7.125,0.7125,-29.980
2023-01-01 00:45:00,4.565807,6.565338,1.0,4.497343,8.994686,4.497343,8.1850,0.401328,0.401328,0.000000,0.000000,3.333333,1.666667,1.0,0.0,6.500,0.6500,-30.000
2023-01-01 01:00:00,4.193034,7.315662,1.0,4.495084,8.990167,4.495084,8.1675,1.000000,1.000000,0.171013,0.153912,3.333333,1.666667,1.0,0.0,5.875,0.5875,26.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 22:00:00,8.248591,35.318144,1.0,3.911441,7.822883,3.911441,2.9700,1.000000,1.000000,1.111111,1.000000,3.333333,1.666667,1.0,0.0,1.000,0.1000,49.790
2023-12-31 22:15:00,7.838805,35.318144,1.0,3.913397,7.826793,3.913397,2.9900,1.000000,1.000000,1.111111,1.000000,3.333333,1.666667,1.0,0.0,1.000,0.1000,-0.070
2023-12-31 22:30:00,7.376144,35.318144,1.0,3.913886,7.827772,3.913886,2.9950,1.000000,1.000000,1.111111,1.000000,3.333333,1.666667,1.0,0.0,1.000,0.1000,-9.900


In [150]:
results.head(3)

,P[W],irradiance_Wm2,ambient_temp_C,P[kWh],elec_price_EUR_per_MWh,gas_price_EUR_per_MWh,dt_h,flats_electricity_needed,flats_heat_needed,flats_gas_consumed,flats_electricity_imported,flats_electricity_selfconsumed,flats_electricity_exported
2023-01-01 00:00:00,0.0,0.0,8.2825,0.0,75.01,14.123013,0.25,7.167999,6.565338,3.333333,3.667999,0.0,0.0
2023-01-01 00:15:00,0.0,0.0,8.2200,0.0,-29.95,14.123013,0.25,6.758386,6.565338,3.333333,3.258386,0.0,0.0
2023-01-01 00:30:00,0.0,0.0,8.2025,0.0,-29.98,14.123013,0.25,6.356152,6.565338,3.333333,2.856152,0.0,0.0


In [148]:
import plotly.graph_objects as go

df = df_results_flats
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=df["electricity_needed_kW"], 
    mode='lines', 
    name="Electricity Needed (kW)",
    yaxis="y1"
))
fig.add_trace(go.Scatter(
    y=df["pv_generation_kWh"], 
    mode='lines', 
    name="PV Generation (kWh)",
    yaxis="y1"
))
fig.add_trace(go.Scatter(
    y=df["battery_soc_history_percent"], 
    mode='lines', 
    name="Battery SOC (%)",
    yaxis="y2"
))

fig.update_layout(
    title="Electricity Needed vs PV Generation with Battery SOC",
    xaxis_title="Timestep",
    yaxis=dict(title="Value (kW / kWh)", side="left"),
    yaxis2=dict(
        title="Battery SOC (%)",
        overlaying="y",
        side="right"
    )
)
fig.show()

In [140]:
flats

In [139]:
results.describe()

,P[W],irradiance_Wm2,ambient_temp_C,P[kWh],elec_price_EUR_per_MWh,gas_price_EUR_per_MWh,dt_h,flats_electricity_needed,flats_heat_needed,flats_gas_consumed,flats_electricity_imported,flats_electricity_selfconsumed,flats_electricity_exported
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.00000,35040.000000,35040.00,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,130.792334,96.269889,10.951336,0.032698,95.80332,44.623157,0.25,9.060515,25.527053,4.018078,4.808284,2.858950,0.530184
std,212.234139,196.177465,8.048102,0.053059,61.21929,15.853599,0.00,3.098564,21.544489,0.684338,4.910885,4.358687,1.680447
min,0.000000,0.000000,-11.980000,0.000000,-149.99000,-1.593696,0.25,0.000000,0.000000,0.000000,-12.791196,0.000000,0.000000
25%,0.000000,0.000000,4.782500,0.000000,63.63000,36.583777,0.25,6.115061,6.869193,3.333333,2.390706,0.000000,0.000000
50%,0.383750,0.000000,10.410000,0.000096,95.10000,45.260883,0.25,9.748917,18.991572,4.444444,4.601581,0.009594,0.000000
75%,181.955000,69.845000,17.021250,0.045489,126.04000,52.785652,0.25,11.144972,40.047856,4.444444,8.481916,4.548875,0.000000
max,880.970000,948.620000,33.410000,0.220243,800.00000,96.317700,0.25,16.125712,114.873703,4.444444,15.091343,18.393059,12.791196


In [ ]:
for key in results:
    # print(f"{key}: {results[key][:5]} ... {results[key][-5:]}")
    print(f"{key}: ", end="")
    print(len(results[key]))

In [ ]:
["chp_gas_consumption_kWh"].plot()

In [ ]:
pd.DataFrame(results)["gas_boiler_gas_consumption_kWh"].plot()

In [ ]:
import pandas as pd

df_gas_boiler = pd.DataFrame({'gas_boiler_gas_consumption_kWh': flats.gas_boiler_gas_consumption_kWh})
df_gas_boiler.head()

In [ ]:
df_gas_boiler.plot()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results.index,
    y=results["ambient_temp_C"],
    name="Ambient Temp (°C)",
    yaxis="y1"
))
fig.add_trace(go.Scatter(
    x=results.index,
    y=results["flats_gas_consumed"],
    name="Flats Gas Consumed (kW)",
    yaxis="y2"
))
fig.update_layout(
    title="Ambient Temperature and Flats Gas Consumed",
    xaxis_title="Time",
    yaxis=dict(title="Ambient Temp (°C)", side="left"),
    yaxis2=dict(title="Gas Consumed (kW)", overlaying="y", side="right"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
)
fig.show()